In [ ]:
from holonote.annotate import Annotator, SQLiteDB
import hvplot.pandas
import pandas as pd
import panel as pn
pn.extension()

## Load data and plot

In [ ]:
speed_data = pd.read_parquet("/Users/droumis/src/holonote/examples/assets/example.parquet")
curve = speed_data.hvplot("TIME", "SPEED", responsive=True, min_height=300)
curve

## Create an annotator object

In [ ]:
annotator = Annotator(
    curve,
    fields=["category"],
    connector=SQLiteDB(table_name="styling"),
)
annotator.groupby = "category"

In [ ]:
from holonote.app.panel import PanelWidgets
from holonote.app.tabulator import AnnotatorTabulator

annotator_widgets = pn.Column(PanelWidgets(annotator), AnnotatorTabulator(annotator))
# could also set to pop up widget with `as_popup=True`

## Create DF and programmatically add annotations

In [ ]:
add_annotations_programmatically = False
if add_annotations_programmatically:
    start_time = pd.date_range("2022-06-04", "2022-06-22", periods=5)
    end_time = start_time + pd.Timedelta(days=2)
    data = {
        "start_time": start_time,
        "end_time": end_time,
        "category": ["Good", "Check", "Good", "Bad", "Good"],
    }
    df = pd.DataFrame(data)
    display(df.head(3))
    annotator.define_annotations(df, TIME=("start_time", "end_time"))

## Display data and annoations together

In [ ]:
pn.Row(annotator_widgets, annotator * curve)

In [ ]:
# annotator.add_annotation(category='NoData')

## Persist annotations to disk programmatically

In [ ]:
annotator.commit()

# 2D Map Air Tempurature Example

In [ ]:
from holonote.annotate import Annotator
from holonote.app import PanelWidgets
import xarray as xr
import panel as pn
import holoviews as hv

hv.extension("bokeh")

ds = xr.tutorial.open_dataset("air_temperature")


def plot_image(time):
    image = hv.Image(ds.sel(time=time), ["lon", "lat"], ["air"]).opts(
        cmap="RdBu_r", title=time, colorbar=True, clim=(220, 340), width=500, height=300
    )
    return image


def plot_timeseries_by_select(indices):
    if indices:
        row = annotator.df.loc[indices[0]]
        lon1 = row["start[lon]"]
        lon2 = row["end[lon]"]
        lat1 = row["start[lat]"]
        lat2 = row["end[lat]"]
        ds_sel = ds.sel(lon=slice(lon1, lon2), lat=slice(lat2, lat1)).mean(
            ["lat", "lon"]
        )
        time_series.object = hv.Curve(ds_sel["air"]).opts(
            title=f"Time Series {lon1:.2f} {lat1:.2f} {lon2:.2f} {lat2:.2f}", width=500
        )


def plot_timeseries_by_stream(bounds):
    if not bounds:
        lon1, lat1, lon2, lat2 = ds.lon.min(), ds.lat.min(), ds.lon.max(), ds.lat.max()
        ds_sel = ds
    else:
        lon1, lat1, lon2, lat2 = bounds
        ds_sel = ds.sel(lon=slice(lon1, lon2), lat=slice(lat2, lat1))
    time_series.object = hv.Curve(ds_sel["air"].mean(["lat", "lon"])).opts(
        title=f"Time Series {lon1:.2f} {lat1:.2f} {lon2:.2f} {lat2:.2f}", width=500
    )


times = ds.time.dt.strftime("%Y-%m-%d %H:%M").values.tolist()

# start annotation
annotator = Annotator({"lon": float, "lat": float}, fields=["Description", "Time", "Z"])
annotator_widgets = PanelWidgets(
    annotator, field_values={"Time": times, "Z": 100}, as_popup=True
)

# make image dependent on the selected time
time_input = annotator_widgets.fields_widgets[1]
time_input.value = times[0]
image = hv.DynamicMap(pn.bind(plot_image, time_input))

time_series = pn.pane.HoloViews()

# update plot time when a new box is SELECTED
pn.bind(plot_timeseries_by_select, annotator.param.selected_indices, watch=True)

# update plot time when a new box is CREATED
display = annotator.get_display("lat", "lon")
box_stream = display._edit_streams[0]  # to make public later
box_stream.source = image
pn.bind(plot_timeseries_by_stream, box_stream.param.bounds, watch=True)

# layout
pn.Row(annotator_widgets, pn.Column(annotator * image, time_series))


# Simple Example without Table

In [ ]:
import hvplot.pandas
import pandas as pd
from holonote.annotate import Annotator, SQLiteDB
from holonote.app import PanelWidgets
import panel as pn

# Setup
speed_data = pd.read_parquet("/Users/droumis/src/holonote/examples/assets/example.parquet")
curve = speed_data.hvplot("TIME", "SPEED")

annotator = Annotator(
    curve,
    fields=["category"],
    connector=SQLiteDB(table_name="styling"),
)

start_time = pd.date_range("2022-06-04", "2022-06-22", periods=5)
end_time = start_time + pd.Timedelta(days=2)
data = {
    "start_time": start_time,
    "end_time": end_time,
    "category": ["A", "B", "A", "C", "B"],
}
annotator.define_annotations(pd.DataFrame(data), TIME=("start_time", "end_time"))

# News stuff
annotator.groupby = "category"
pn.Row(PanelWidgets(annotator), annotator * curve)

# Copy Paste Example 

In [ ]:
import hvplot.pandas
import pandas as pd
from holonote.annotate import Annotator, SQLiteDB
from holonote.app import PanelWidgets
import panel as pn
from holonote.app.tabulator import AnnotatorTabulator

# Setup
speed_data = pd.read_parquet("/Users/droumis/src/holonote/examples/assets/example.parquet")
curve = speed_data.hvplot("TIME", "SPEED")

annotator = Annotator(
    curve,
    fields=["category"],
    connector=SQLiteDB(table_name="styling"),
)

start_time = pd.date_range("2022-06-04", "2022-06-22", periods=5)
end_time = start_time + pd.Timedelta(days=2)
data = {
    "start_time": start_time,
    "end_time": end_time,
    "category": ["A", "B", "A", "C", "B"],
}
annotator.define_annotations(pd.DataFrame(data), TIME=("start_time", "end_time"))

annotator.groupby = "category"
annotator_widgets = pn.Column(PanelWidgets(annotator), AnnotatorTabulator(annotator))
pn.Row(annotator_widgets, annotator * curve)